In [110]:
import pandas as pd
from datetime import datetime as dt , date , timedelta
from dateutil.relativedelta import relativedelta

In [111]:
deuda = []

bases = ['30/360' , 'ACT/365' , 'ACT/ACT']
sistemas = ['Aleman', 'Frances', 'Americano', 'Bullet']
periodicidades = ['Mensual', 'Bimensual', 'Trimestral', 'Cuatrimestral', 'Semestral', 'Anual']
tipos_tasas = ['TNA' , 'TEA' , 'TEM']
# codigo es el número identificatorio del préstamo, fecha es el día del desembolso inicial, base es la convención utilizada para la cuenta de días, sistema es la forma en la cual se
# amortiza el capital, periodicidad es el plazo entre dos amortizaciones, gracia es si existe un período de gracia, capital es el importe total del préstamo, tasa es la tasa pactada
# tipo de tasa es la tasa informada por el banco, plazo es [undefined]

class instrumento():
  def __init__(self, codigo, fecha, base, sistema, periodicidad, amortizaciones, gracia, capital, tasa, tipo_tasa, plazo):
    self.codigo = codigo
    if base not in bases:
      raise ValueError(f"Seleccione una base válida dentro de las siguientes {bases}")
    self.fecha = dt.strptime(str(fecha), "%d/%m/%Y")
    self.base = base
    if sistema not in sistemas:
      raise ValueError(f"Seleccione un sistema válido dentro de los siguientes {sistemas}")
    self.sistema = sistema
    if periodicidad not in periodicidades:
      raise ValueError(f"Seleccione una periodicidad válida dentro de las siguientes {periodicidades}")
    self.periodicidad = periodicidad
    try:
      self.sistema != 'Bullet'
    except:
      self.amortizaciones = 1
    finally:
      self. amortizaciones = amortizaciones
    self.gracia = None
    self.capital = capital
    self.tasa = tasa
    if tipo_tasa not in tipos_tasas:
      raise ValueError(f"Seleccione un tipo de tasa válido dentro de las siguientes {tipos_tasas}")
    self.tipo_tasa = tipo_tasa
    self.plazo = plazo
    # definición inicial de parámetros

    if self.periodicidad == 'Mensual':
      self.meses = 1
    elif self.periodicidad == 'Bimensual':
      self.meses = 2
    elif self.periodicidad == 'Trimestral':
      self.meses = 3
    elif self.periodicidad == 'Cuatrimestral':
      self.meses = 4
    elif self.periodicidad == 'Semestral':
      self.meses = 6
    else:
      self.meses = 12
    # para las diferentes periodicidades se calculan los meses que transcurren entre amortizaciones

  def TEM(self):
    if self.tipo_tasa == 'TEM':
      return self.tasa
    elif self.tipo_tasa == 'TNA':
      return (self.tasa/12)
    else:
      return (1+self.tasa)**(1/12) - 1

  def TNA(self):
    if self.tipo_tasa == 'TNA':
      return self.tasa
    elif self.tipo_tasa == 'TEM':
      return (self.tasa * 12)
    else:
      return (((1+self.tasa)**(1/12))-1)*12

  def TEA(self):
    if self.tipo_tasa == 'TEA':
      return self.tasa
    elif self.tipo_tasa == 'TNA':
      return ((1+(self.tasa/12))**12)-1
    else:
      return ((1+self.tasa)**12)-1

  def get_tasa(self):
    if self.tipo_tasa == 'TEA':
      return print(f"El instrumento {self.codigo} fue pactado a {self.tasa * 100: .4f}% {self.tipo_tasa}, lo que equivale a {self.TEM() * 100: .4f}% TEM y {self.TNA() * 100: .4f}% TNA")
    elif self.tipo_tasa == 'TNA':
      return print(f"El instrumento {self.codigo} fue pactado a {self.tasa * 100: .4f}% {self.tipo_tasa}, lo que equivale a {self.TEM() * 100: .4f}% TEM y {self.TEA() * 100: .4f}% TEA")
    else:
      return print(f"El instrumento {self.codigo} fue pactado a {self.tasa * 100: .4f}% {self.tipo_tasa}, lo que equivale a {self.TNA() * 100: .4f}% TNA y {self.TEA() * 100: .4f}% TEA")

    # Se incluye una pequeña calculadora de conversión de tasas entre TEM, TNA y TEA

  def cuadro_pago(self):
    filas = [{'cuota': 0, 'nro_prestamo': self.codigo, 'fecha': self.fecha, 'capital_rest': self.capital , 'amortizacion': self.capital, 'interes': None, 'total_cuota': self.capital}]
    if self.sistema == 'Frances': # El sistema Francés es aquel en el que las cuotas son constantes, siendo variable el interés y la amortización
      for i in range(1, self.amortizaciones + 1):
        codigo = self.codigo
        if self.base == 'ACT/ACT' or self.base == 'ACT/365':
          if self.periodicidad == 'Anual':
            end_period = self.fecha + relativedelta(months=12 * i)
          elif self.periodicidad == 'Semestral':
            end_period = self.fecha + relativedelta(months=6 * i)
          elif self.periodicidad == 'Cuatrimestral':
            end_period = self.fecha + relativedelta(months=4 * i)
          elif self.periodicidad == 'Trimestral':
            end_period = self.fecha + relativedelta(months=3 * i)
          elif self.periodicidad == 'Bimensual':
            end_period = self.fecha + relativedelta(months=3 * i)
          else:
            end_period = self.fecha + relativedelta(months=1 * i)
            if self.base == 'ACT/ACT' and end_period.month == 2 and not (end_period.year % 4 == 0 and (end_period.year % 100 != 0 or end_period.year % 400 == 0)):
              end_period = end_period.replace(day=28)
          fecha = end_period
        else:
          end_period = self.fecha + relativedelta(days = i * 30 * self.meses)
        fecha = end_period
        total_cuota = round(-self.capital * ((self.TEM() * (1+ self.TEM()) ** self.amortizaciones) / ((1 + self.TEM()) ** self.amortizaciones - 1)) , 2)
        interes = round(-(filas[-1]['capital_rest'] * self.TEM()) , 2)
        amortizacion = round(total_cuota - interes , 2)
        capital_rest = round(amortizacion + filas[-1]['capital_rest'] , 2)
        fila = {'cuota': i , 'nro_prestamo': codigo , 'fecha': fecha, 'capital_rest': capital_rest, 'amortizacion': amortizacion , 'interes': interes ,  'total_cuota': total_cuota }
        filas.append(fila)
      instrumento_df = pd.DataFrame(filas)
      return instrumento_df
    elif self.sistema == 'Aleman': # En el sistema Alemán, la amortización es constante, por ende el interés y la cuota total varían en cada pago
      for i in range(1, self.amortizaciones + 1):
        codigo = self.codigo
        if self.base == 'ACT/ACT' or self.base == 'ACT/365':
          if self.periodicidad == 'Anual':
            end_period = self.fecha + relativedelta(months=12 * i)
          elif self.periodicidad == 'Semestral':
            end_period = self.fecha + relativedelta(months=6 * i)
          elif self.periodicidad == 'Cuatrimestral':
            end_period = self.fecha + relativedelta(months=4 * i)
          elif self.periodicidad == 'Trimestral':
            end_period = self.fecha + relativedelta(months=3 * i)
          elif self.periodicidad == 'Bimensual':
            end_period = self.fecha + relativedelta(months=3 * i)
          else:
            end_period = self.fecha + relativedelta(months=1 * i)
            if self.base == 'ACT/ACT' and end_period.month == 2 and not (end_period.year % 4 == 0 and (end_period.year % 100 != 0 or end_period.year % 400 == 0)):
              end_period = end_period.replace(day=28)
          fecha = end_period
        else:
          end_period = self.fecha + relativedelta(days = i * 30 * self.meses)
          fecha = end_period
        amortizacion = round(-self.capital / self.amortizaciones , 2)
        capital_rest = round(amortizacion + filas[-1]['capital_rest'] , 2)
        days_in_period = (end_period - filas[-1]['fecha']).days
        interes = round(-(filas[-1]['capital_rest'] * (self.TNA() / 365 * days_in_period)), 2)
        total_cuota = round(amortizacion + interes , 2)
        fila = {'cuota': i , 'nro_prestamo': codigo , 'fecha': fecha, 'capital_rest': capital_rest, 'amortizacion': amortizacion , 'interes': interes ,  'total_cuota': total_cuota }
        filas.append(fila)
      instrumento_df = pd.DataFrame(filas)
      return instrumento_df
    elif self.sistema == 'Bullet': # El Bullet es un préstamo en el cuál la amortización se realiza íntegramente en un pago al final del período
      if self.plazo is None:
        raise ValueError("Para un préstamo Bullet debe especificar el plazo pactado")
      if self.amortizaciones > 1:
        raise ValueError(f"En un préstamo Bullet las amortizaciones no pueden ser más que 1. Usted ingresó {self.amortizaciones}")
      else:
        for i in range(1, self.amortizaciones + 1):
          codigo = self.codigo
          end_period = self.fecha + relativedelta(days = self.plazo)
          fecha = end_period
          amortizacion = -self.capital
          days_in_period = (end_period - filas[-1]['fecha']).days
          capital_rest = round(amortizacion + filas[-1]['capital_rest'] , 2)
          interes = round(-(filas[-1]['capital_rest'] * (self.TNA() / 365 * days_in_period)), 2)
          total_cuota = round(amortizacion + interes , 2)
          fila = {'cuota': i , 'nro_prestamo': codigo , 'fecha': fecha, 'capital_rest': capital_rest, 'amortizacion': amortizacion , 'interes': interes ,  'total_cuota': total_cuota }
          filas.append(fila)
      instrumento_df = pd.DataFrame(filas)
      return instrumento_df
    else:
      pass

In [160]:
# Definiremos algunos préstamos de pueba con distintos parámetros:

frances1 = instrumento('frances1' , '10/12/2022' , '30/360' , 'Frances' , 'Mensual' , 12 , None , 1000000 , 0.6 , 'TNA' , None)
frances2 = instrumento('frances2' , '28/02/2023' , 'ACT/ACT' , 'Frances' , 'Trimestral' , 12 , None , 945000 , 0.7 , 'TEA' , None)
aleman1 = instrumento('aleman1' , '31/08/2022' , 'ACT/ACT' , 'Aleman' , 'Semestral' , 8 , None , 114781326 , 0.62 , 'TNA' , None)
aleman2 = instrumento('aleman2' , '22/02/2022' , 'ACT/ACT' , 'Aleman' , 'Mensual' , 24 , None , 5650000 , 0.5 , 'TNA' , None)
bullet1 = instrumento('bullet1' , '06/09/2023' , 'ACT/ACT' , 'Bullet' , 'Semestral' , 1 , None , 315000000 , 1.68 , 'TNA' , 147)

In [152]:
# Probemos primero la calculadora de tasas:

frances2.get_tasa()

El instrumento prestamo fue pactado a  70.0000% TEA, lo que equivale a  4.5211% TEM y  54.2535% TNA


In [161]:
# Tomamos alguno de los ejemplos y probamos el método 'cuadro_pago()' :

aleman1.cuadro_pago()

,cuota,nro_prestamo,fecha,capital_rest,amortizacion,interes,total_cuota
0,0,aleman1,2022-08-31,1.147813e+08,1.147813e+08,NaN,1.147813e+08
1,1,aleman1,2023-02-28,1.004337e+08,-1.434767e+07,-35289754.53,-4.963742e+07
2,2,aleman1,2023-08-31,8.608599e+07,-1.434767e+07,-31390334.14,-4.573800e+07
3,3,aleman1,2024-02-29,7.173833e+07,-1.434767e+07,-26613544.16,-4.096121e+07
4,4,aleman1,2024-08-31,5.739066e+07,-1.434767e+07,-22421667.24,-3.676933e+07
5,5,aleman1,2025-02-28,4.304300e+07,-1.434767e+07,-17644877.27,-3.199254e+07
6,6,aleman1,2025-08-31,2.869533e+07,-1.434767e+07,-13453000.35,-2.780067e+07
7,7,aleman1,2026-02-28,1.434767e+07,-1.434767e+07,-8822438.63,-2.317010e+07
8,8,aleman1,2026-08-31,0.000000e+00,-1.434767e+07,-4484333.45,-1.883200e+07


In [162]:
# Podemos modificar manualmente datos de la DF. Para ello definimos la variable y procedemos a modificar los datos que necesitamos:

aleman1bis = aleman1.cuadro_pago()
aleman1bis['fecha'][0] = '2022-09-05'
aleman1bis['interes'][1] = -34314899.43
aleman1bis['total_cuota'][1] = -45737999.89
aleman1bis

<ipython-input-162-3a090bc023cb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aleman1bis['fecha'][0] = '2022-09-05'
<ipython-input-162-3a090bc023cb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aleman1bis['interes'][1] = -34314899.43
<ipython-input-162-3a090bc023cb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aleman1bis['total_cuota'][1] = -45737999.89


,cuota,nro_prestamo,fecha,capital_rest,amortizacion,interes,total_cuota
0,0,aleman1,2022-09-05,1.147813e+08,1.147813e+08,NaN,1.147813e+08
1,1,aleman1,2023-02-28,1.004337e+08,-1.434767e+07,-34314899.43,-4.573800e+07
2,2,aleman1,2023-08-31,8.608599e+07,-1.434767e+07,-31390334.14,-4.573800e+07
3,3,aleman1,2024-02-29,7.173833e+07,-1.434767e+07,-26613544.16,-4.096121e+07
4,4,aleman1,2024-08-31,5.739066e+07,-1.434767e+07,-22421667.24,-3.676933e+07
5,5,aleman1,2025-02-28,4.304300e+07,-1.434767e+07,-17644877.27,-3.199254e+07
6,6,aleman1,2025-08-31,2.869533e+07,-1.434767e+07,-13453000.35,-2.780067e+07
7,7,aleman1,2026-02-28,1.434767e+07,-1.434767e+07,-8822438.63,-2.317010e+07
8,8,aleman1,2026-08-31,0.000000e+00,-1.434767e+07,-4484333.45,-1.883200e+07


In [163]:
# Por último, si la calculadora funciona y cada préstamo se confirma, podemos ir guardando los mismos en una DF que acumule toda la deuda:

frances1 = frances1.cuadro_pago()
deuda_prueba = [frances1 , aleman1bis]
deuda_prueba = pd.concat(deuda_prueba, axis = 0)
deuda_prueba

,cuota,nro_prestamo,fecha,capital_rest,amortizacion,interes,total_cuota
0,0,frances1,2022-12-10,1.000000e+06,1.000000e+06,NaN,1.000000e+06
1,1,frances1,2023-01-09,9.371746e+05,-6.282541e+04,-50000.00,-1.128254e+05
2,2,frances1,2023-02-08,8.712079e+05,-6.596668e+04,-46858.73,-1.128254e+05
3,3,frances1,2023-03-10,8.019429e+05,-6.926501e+04,-43560.40,-1.128254e+05
4,4,frances1,2023-04-09,7.292146e+05,-7.272827e+04,-40097.14,-1.128254e+05
5,5,frances1,2023-05-09,6.528499e+05,-7.636468e+04,-36460.73,-1.128254e+05
6,6,frances1,2023-06-08,5.726670e+05,-8.018291e+04,-32642.50,-1.128254e+05
7,7,frances1,2023-07-08,4.884750e+05,-8.419206e+04,-28633.35,-1.128254e+05
8,8,frances1,2023-08-07,4.000733e+05,-8.840166e+04,-24423.75,-1.128254e+05
9,9,frances1,2023-09-06,3.072516e+05,-9.282174e+04,-20003.67,-1.128254e+05
